In [1]:
%load_ext autoreload
%autoreload 2

from scribbles_creator import *
from PIL import Image
import napari
import numpy as np
import os

## Create scribble annotation from an image mask of the cell pose dataset

In [35]:
def get_cellpose_img_data(folder_path, img_num, load_img=False, load_gt=False, load_scribbles=False, mode="NA", bin="NA", suff=False, load_pred=False, pred_tag="convpaint"):
    
    folder_path = folder_path if folder_path[-1] == "/" else folder_path + "/"
    img_base = str(img_num).zfill(3)

    img_path = folder_path + img_base + f"_img.png"
    if load_img:
        img = np.array(Image.open(img_path))
    else:
        img = None

    gt_path = folder_path + img_base + "_ground_truth.png"
    if load_gt:
        ground_truth = np.array(Image.open(gt_path))
    else:
        ground_truth = None

    suff = "" if not suff else "_" + suff

    scribbles_path = folder_path + img_base + f"_scribbles_{mode}_{bin}{suff}.png"
    if load_scribbles:
        scribbles = np.array(Image.open(scribbles_path))
    else:
        scribbles = None

    pred_path = folder_path + img_base + f"_{pred_tag}_{mode}_{bin}{suff}.png"
    if load_pred:
        pred = np.array(Image.open(pred_path))  
    else:
        pred = None

    img_data = {"img_path": img_path, "gt_path": gt_path, "scribbles_path": scribbles_path, "pred_path": pred_path,
                "img": img, "gt": ground_truth, "scribbles": scribbles, "pred": pred}

    return img_data

In [37]:
def create_cellpose_gt(folder_path, img_num, save_res=True):
    folder_path = folder_path if folder_path[-1] == "/" else folder_path + "/"
    img_base = str(img_num).zfill(3)
    ground_truth = np.array(Image.open(folder_path + img_base + "_masks.png"))
    # Summarize all non-background classes into one class (we are testing semantic segmentation not instance segmentation)
    ground_truth[ground_truth>0] = 2
    # Set the background class to 1 (since the scribble annotation assumes class 0 to represent non-annotated pixels)
    ground_truth[ground_truth==0] = 1

    if save_res:
        if not os.path.exists(gt_path):
            gt_path = folder_path + img_base + "_ground_truth.png"
            gt_img = Image.fromarray(ground_truth)
            gt_img.save(gt_path)
    
    return ground_truth


In [38]:
def create_cellpose_scribble(folder_path, img_num, bin=0.2, sq_scaling=False, mode="all", save_res=False, suff=False, show_res=False):

    # Load the ground truth and get the scribbles path for saving; note that if we want to show the results, we also load the image
    img_data = get_cellpose_img_data(folder_path, img_num, load_img=show_res, load_gt=True, mode=mode, bin=bin, suff=suff)
    ground_truth = img_data["gt"]
    scribbles_path = img_data["scribbles_path"]
    # Create the scribbles
    scribbles = create_even_scribble(ground_truth, max_perc=bin, sq_scaling=False, mode=mode)
    perc_labelled = np.sum(scribbles>0) / (scribbles.shape[0] * scribbles.shape[1]) * 100

    if save_res:
        # Save the scribble annotation as an image
        scribble_img = Image.fromarray(scribbles)
        scribble_img.save(scribbles_path)

    if show_res:
        # Show the image, ground truth and the scribbles
        image = img_data["img"]
        v = napari.Viewer()
        v.add_image(image)
        v.add_labels(ground_truth)
        v.add_labels(scribbles)

    return scribbles, perc_labelled

In [40]:
folder_path = "./imgs/train"
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:1]
percentages = {}
for bin in [0.01]:#, 0.025, 0.05, 0.1, 0.25, 0.5, 1]:
    percentages[bin] = []
    for img_num in range(0, 1):
        for s in suff:
            scribbles, perc_labelled = create_cellpose_scribble(folder_path, img_num, bin=bin, sq_scaling=False, mode = "all", show_res=True, suff=s, save_res=True)
            percentages[bin].append(perc_labelled)

In [41]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 0.01: mean = 0.0092, std = 0.0000, range = 0.0092 - 0.0092


In [ ]:
v = napari.Viewer()
v.add_labels(scribbles)

## Load and show scribbles and ground truth

In [44]:
# Show the image, ground truth and the scribble annotation in a napari viewer
img_num = 0
mode = "all"
bin = 0.01
suff = "a"
folder_path = "./imgs/train"
img_data = get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=True, load_scribbles=True, mode=mode, bin=bin, suff=suff)
v = napari.Viewer()
v.add_image(img_data["img"], name="image")
v.add_labels(img_data["gt"], name="ground truth")
v.add_labels(img_data["scribbles"], name="scribbles")

<Labels layer 'scribbles' at 0x219006d2ac0>

## When we have performed segmentation in the Napari plugin, we can calculate its accuracy

In [ ]:
segmentation = v.layers['segmentation'].data
acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
print(f"Accuracy (pixels correctly segmented): {acc}%")